## выкачивание данных

In [3]:
import overpy
op = overpy.Overpass()

In [123]:
# http://gis.stackexchange.com/questions/178424/overpass-turbo-area-code-lookup
AREA = "3600060189"  # Russia

In [41]:
r = op.query('''
area({})->.searchArea;
(
    way
    ["name"]["highway"]
    (area.searchArea);
);
out;
'''.format(AREA))

In [47]:
import cPickle
with open('ways_ru.pcl', 'wb') as pcl:
    cPickle.dump(r, pcl)

## загрузка данных

In [ ]:
import cPickle
with open('ways_ru.pcl', 'rb') as pcl:
    r = cPickle.load(pcl)

In [136]:
r.ways[1].tags

{'highway': 'tertiary',
 'name': u'\u041a\u043e\u043c\u0441\u043e\u043c\u043e\u043b\u044c\u0441\u043a\u0430\u044f \u0443\u043b\u0438\u0446\u0430',
 'name:en': "Komsomol'skaja street",
 'name:ru': u'\u041a\u043e\u043c\u0441\u043e\u043c\u043e\u043b\u044c\u0441\u043a\u0430\u044f \u0443\u043b\u0438\u0446\u0430',
 'postal_code': '183038',
 'surface': 'asphalt'}

In [134]:
import pandas as pd

df = pd.DataFrame(
    data=[w.tags for w in r.ways],
    index=[w.id for w in r.ways],
    columns=['name', 'highway', 'name:en', 'name:de']
)
df.head()

,name,highway,name:en,name:de
4078548,Шепетовская улица,residential,NaN,NaN
4391345,Комсомольская улица,tertiary,Komsomol'skaja street,NaN
4391346,проспект Ленина,primary,Lenina avenue,NaN
4391347,улица Полярные Зори,secondary,Poljarnye Zori street,NaN
4391349,улица Книповича,secondary,Knipovicha street,NaN


In [135]:
len(df)

659126

In [68]:
df.name.value_counts().head()

Советская улица      8672
улица Ленина         8444
Центральная улица    6983
Школьная улица       5531
Садовая улица        5492
Name: name, dtype: int64

In [128]:
streets_of = df[
    df.name.str.startswith(u'улица')
    | df.name.str.startswith(u'проспект')
]
streets_of.name.value_counts().head(15)

улица Ленина          8444
улица Мира            5095
улица Гагарина        3883
улица Кирова          3530
улица Пушкина         3012
улица Калинина        2843
улица Карла Маркса    2302
улица Горького        2209
улица Чапаева         2084
улица Победы          2019
улица Строителей      1725
улица Дзержинского    1645
улица Чкалова         1508
проспект Ленина       1485
улица Свердлова       1476
Name: name, dtype: int64

In [151]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

streets_of.name.map(
    lambda s: morph.normal_forms(s.split()[-1])[0]
).value_counts().head(10)

ленин      9946
мир        5830
гагарин    4194
победа     3964
киров      3758
горький    3079
пушкин     3028
калинин    2938
маркс      2672
октябрь    2499
Name: name, dtype: int64